# Application des RNN sur les données de news

Essayez de créer un RNN sur les données de News étudiée dans la première partie.

# Classifying newswires: a multi-class classification example


## The Reuters dataset

Nous allons travailler avec _Reuters dataset_, un ensemble de courts fils de presse et leurs sujets, publiés par Reuters en 1986. Il y a 46 sujets différents; certains sujets sont plus représentés que d'autres, mais chacun des sujets contient au moins 10 exemples dans l'ensemble d'apprentissage.

Comme pour IMDB et MNIST, le jeu de données Reuters est livré avec Keras. Jetons un coup d'oeil tout de suite:

In [3]:
from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

2113536/2110848 [==============================] - 3s 2us/step


Les données ressemblent à celle de IMDB.

On peut rencoder les phrases avec :

In [4]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# Note that our indices were offset by 3
# because 0, 1 and 2 are reserved indices for "padding", "start of sequence", and "unknown".
decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])

557056/550378 [==============================] - 1s 2us/step


In [7]:
decoded_newswire

'? ? ? said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3'

## Préparation des données

In [5]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

# Our vectorized training data
x_train = vectorize_sequences(train_data)
# Our vectorized test data
x_test = vectorize_sequences(test_data)

In [6]:
def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results

# Our vectorized training labels
one_hot_train_labels = to_one_hot(train_labels)
# Our vectorized test labels
one_hot_test_labels = to_one_hot(test_labels)

In [11]:
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

## Construire notre réseau

Ce problème de classification par sujet est très similaire à notre problème de classification précédent dans les critiques de films: dans les deux cas, nous essayons de:
- classer des extraits courts de texte. Il existe toutefois une nouvelle contrainte: le nombre de classes de sortie est passé de 2 à 46, c’est-à-dire que la dimensionnalité de l'espace de sortie est beaucoup plus grande.

- Dans une pile de couches `Dense` comme celle que nous utilisions, chaque couche ne peut accéder qu’aux informations présentes dans la sortie de la couche précédente.

- Si une couche supprime des informations relatives au problème de classification, ces informations ne pourront jamais être récupérées par les couches suivantes:

- L'espace 16 dimensions peut être trop limité pour apprendre à séparer 46 classes différentes: de telles petites couches peuvent agir comme des goulots d'étranglement, laisser tomber en permanence des informations pertinentes.

Pour cette raison, nous utiliserons des couches plus grandes.